# Option Pricing

This notebook demonstrates how to price Options using the Instrument Pricing Analytics (IPA) and the Refinitiv Data Libraries.
##### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
 and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 


## Import the library and connect to Refinitiv Data Libraries

In [1]:
import refinitiv.data as rd
import refinitiv.data.content.ipa.financial_contracts as rdf
from refinitiv.data.content.ipa.financial_contracts import option

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x1060110a0 {name='workspace'}>

## Option Pricing

This sample shows how to model options and compute analytics both for listed and OTC options.

### Single ETI Option

In this sample, we compute the implied volatility and some greeks for an index option. More output fields can be found on the Refinitiv Developer Community [here](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-apis/docs?content=94324&type=documentation_item)

### Multiple ETI Options with a global parameters

In the example below we compute the same analytics but for a list of options. We also set a valuation date in the past globaly and force the use of the Binomial model (Black&Scholes is the default for european options). The full list of pricing parameters can be found on the Refinitiv Developer Community [here](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-apis/docs?content = 94324&type = documentation_item)

In [3]:
response = rdf.Definitions(
    universe=[
        option.Definition(
            underlying_type=option.UnderlyingType.ETI,
            instrument_code="FCHI700000F4.p"),
        option.Definition(
            underlying_type=option.UnderlyingType.ETI,
            instrument_code="FCHI700000F4.p"),
    ],
    pricing_parameters=option.PricingParameters(
        valuation_date="2023-01-05",
        pricing_model_type=option.PricingModelType.BINOMIAL),
    fields=["ValuationDate",
            "OptionType",
            "ExerciseType",
            "ExerciseStyle",
            "EndDate",
            "StrikePrice",
            "VolatilityPercent",
            "DeltaPercent",
            "GammaPercent"]
).get_data()

response.data.df

,ValuationDate,OptionType,ExerciseType,ExerciseStyle,EndDate,StrikePrice,VolatilityPercent,DeltaPercent,GammaPercent
0,2023-01-05,Vanilla,CALL,EURO,2024-06-21,7000,16.536506,0.455178,0.000284
1,2023-01-05,Vanilla,CALL,EURO,2024-06-21,7000,16.536506,0.455178,0.000284


### Global & Individual pricing parameters

In the sample below the two options are analyzed on 2019, July 5th. For the sesond option we specifically use the BID side of the option price instead of the MID, which is the default value. 

In [4]:
response = rdf.Definitions(

    universe=[
        option.Definition(
            underlying_type=option.UnderlyingType.ETI,
            instrument_code="FCHI700000F4.p"),

        option.Definition(
            underlying_type=option.UnderlyingType.ETI,
            instrument_code="FCHI700000F4.p",
            pricing_parameters=option.PricingParameters(
                price_side=option.PriceSide.BID)),
    ],
    pricing_parameters=option.PricingParameters(
        valuation_date="2023-01-05"),

    fields=["ValuationDate",
            "OptionType",
            "ExerciseType",
            "ExerciseStyle",
            "EndDate",
            "StrikePrice",
            "VolatilityPercent",
            "DeltaPercent",
            "GammaPercent"]
).get_data()

response.data.df

,ValuationDate,OptionType,ExerciseType,ExerciseStyle,EndDate,StrikePrice,VolatilityPercent,DeltaPercent,GammaPercent
0,2023-01-05,Vanilla,CALL,EURO,2024-06-21,7000,16.610194,0.455113,0.000281
1,2023-01-05,Vanilla,CALL,EURO,2024-06-21,7000,16.610194,0.455113,0.000281


### OTC FX Option
It is also possible to define an OTC options by providing its definition through parameters. The full list is described on the Refinitiv Developer Community [here](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-apis/docs?content = 89174&type = documentation_item).
In the sample below we describe a 5M vanilla call on AUD / USD starting today with a strike of 0.64 and a notional of 1M


In [5]:
response = option.Definition(

    underlying_type=option.UnderlyingType.FX,
    underlying_definition=option.FxUnderlyingDefinition("AUDUSD"),
    tenor="5M",
    strike=0.6400,
    call_put="Call",
    exercise_style="EURO",
    notional_amount=1000000,
    notional_ccy="AUD",
    fields=["ValuationDate",
            "OptionType",
            "ExerciseType",
            "ExerciseStyle",
            "EndDate",
            "StrikePrice",
            "PremiumPercent",
            "DeltaPercent",
            "GammaPercent"]
).get_data()

response.data.df

,ValuationDate,OptionType,ExerciseType,ExerciseStyle,EndDate,StrikePrice,PremiumPercent,DeltaPercent,GammaPercent
0,2023-03-07,Vanilla,CALL,EURO,2023-08-07,0.64,6.214879,73.872731,3.827358


## Close the default session

In [6]:
rd.close_session()